In [2]:
import threading
from time import sleep, perf_counter

# The innacuracy of `sleep`

In [67]:
def sleep_and_print(t: int):
    """Sleep t seconds and print"""
    sleep(t)
    print(t)

t1 = threading.Thread(target=sleep_and_print, args=(0.00005,))
t2 = threading.Thread(target=sleep_and_print, args=(0.00009,))

t1.start()
t2.start()

9e-05
5e-05


Look at the wrong order here! `0.00009` printed before `0.00005` even though it is a longer wait! 

Another fun problem is that when the sleep times are lower than milliseconds, the code becomes **non-deterministic** since the scheduler choses threads to run based on an arbitrary scheduling algorithm - so the order in which the threads run is up to the OS scheduler and not the actual values :(

# Sleep Sort

In [5]:
## Helper function
def add_item_after_sleep(arr: list[int], item: int, timeout: int):
    """Adds `item` to `arr` after sleeping for `timeout` seconds"""
    sleep(timeout)
    arr.append(item)

In [4]:
def sleep_sort(L: list[int]) -> list[int]:
    """Performs sleep sort on L which is an array of *non-negative* integers"""
    to_ret = []
    
    threads: list[threading.Thread] = []

    for item in L:   # .... looks O(n)??
        t = threading.Thread(target=add_item_after_sleep, args=(to_ret, item, item))
        threads.append(t)
        t.start()
    
    for t in threads:
        t.join()
    
    return to_ret

In [69]:
sleep_sort([5,3,1,2,4])

[1, 2, 3, 4, 5]

## Scaled sleep sort

In [ ]:
def scaled_sleep_sort(L: list[int]) -> list[int]:
    """Performs sleep sort where the sleep times of elements are scaled by max(L)"""
    to_ret = []
    
    threads: list[threading.Thread] = []

    max_item = max(L)  # scale all values down by this so that every sleep is in [0,1]

    for item in L:
        t = threading.Thread(target=add_item_after_sleep, args=(to_ret, item, item / max_item))
        threads.append(t)
        t.start()
    
    for t in threads:
        t.join()
    
    return to_ret

In [80]:
# An example which works (where the lowest sleep is >1ms)
scaled_sleep_sort([5,3,1,2,4])

[1, 2, 3, 4, 5]

In [94]:
# An example which fails (where the lowest sleep is <1ms)
scaled_sleep_sort([10000, 6, 3])

# here the delays should have been 1, 0.0006 and 0.0003, but 6 got added before 3!

[6, 3, 10000]

Mathematically, for `scaled_sleep_sort` to work correctly we require that:
$$
\dfrac{\min(L)}{\max(L)} > 0.001
$$

ie. the ratio of the highest and lowest elements of $L$ must be less than 0.001 (which is 1ms). This is the only condition in which we can guarantee correctness

# Comparing run times

In [8]:
## Measuring pythons in-built sorted function
start = perf_counter()
print("In-built sort result:", sorted([5,3,1,2,4]))
print("In-built sort time:", perf_counter() - start)

print()

## Measuring sleep_sort
start = perf_counter()
print("Sleep sort result:", sleep_sort([5,3,1,2,4]))
print("Sleep sort time:", perf_counter() - start)

print()

## Measuring scaled_sleep_sort for values where it can work (ie. max(L)/min(L) < 0.001)
start = perf_counter()
print("Scaled sleep sort result:", scaled_sleep_sort([5,3,1,2,4]))
print("Scaled sleep sort time:", perf_counter() - start)

In-built sort result: [1, 2, 3, 4, 5]
In-built sort time: 0.00015250200158334337

Sleep sort result: [1, 2, 3, 4, 5]
Sleep sort time: 5.006125771997176

Scaled sleep sort result: [1, 2, 3, 4, 5]
Scaled sleep sort time: 1.0028352759982226


Even in the cases where the scaled down sleep sort works, its too slow compared to any good comparison sorting algorithm! 

**This defeats any practicality of it :(**